In [1]:
#import modules and set up environment 
import os
import sys
path = "../../src/"

sys.path.append(path)
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jax.numpy as jnp
from jax import jit, grad, vmap, hessian, scipy, random

#import sgmcmc code 
import models.linear_regression.linear_regression as lr
from samplers import sgd as sgd
from samplers import sgld as sgld
from samplers import sgldps as sgldps
from samplers import sgldcv as sgldcv
from samplers import sgldcvps as sgldcvps

key = random.PRNGKey(11)

In [2]:
#set up model-specific gradient functions
gradf_0 = lr.gradf_0
gradf_i_batch = lr.gradf_i_batch
post_var = lr.post_var

### Set up model and data

In [3]:
#load in the data
file_path = "../../data/real/casp_scaled.csv"
data = pd.read_csv(file_path)
dat_array = data.values[:]
N = dat_array.shape[0]
x = np.column_stack([np.ones(N), dat_array[:, 1:]])
y = dat_array[:,0]

#set up model parameters
dim = x.shape[1] 

In [4]:
data.head()

,RMSD,F1,F2,F3,F4,F5,F6,F7,F8,F9
0,1.558531,0.908481,0.879584,0.240885,1.058750,0.894440,0.996035,0.149539,0.566885,-1.253153
1,-0.282357,-0.906740,-0.952127,-0.640245,-0.903989,-1.001457,-0.834813,-0.331492,-0.548301,0.672800
2,0.249495,-0.528725,-0.881704,-1.255652,-0.653210,-0.518857,-0.912117,-0.505989,-0.725315,0.717134
3,1.324313,-0.356576,-0.443293,-0.338425,-0.643398,-0.279821,-0.517141,-0.371967,0.000441,0.759477
4,0.034891,-0.594061,-0.874424,-1.106651,-0.921619,-0.615710,-0.730225,-0.589569,-0.512898,0.901558


### Set up sampling framework

In [5]:
# step-size
step_size = 1e-5
step_sgd = 1e-3
# batch sizes
n_batch = np.int64(N*0.001)
# number of chains (first chain discarded due to slower runtime)
N_rep = 11
#iterations
burnin = np.int64(10/0.001)
Niter = 2*burnin

### Running SGLD-CV (0.001N) sampler

In [6]:
runtime_df=[]
samples_df = []
grads_df = []

In [7]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    theta_hat, samples_sgd, runtime_sgd = sgd.adam(key, gradf_0, gradf_i_batch, burnin, theta_0, x, y, n_batch, step_sgd, replacement=True)
    samples, grads, runtime = sgldcv.sgld_cv_sampler(subkey, gradf_0, gradf_i_batch, burnin, step_size, theta_hat, theta_hat, x, y, n_batch, replacement=True)
    
    runtime_df.append(np.concatenate((runtime_sgd, runtime_sgd[burnin-1]+runtime))) #join output
    samples_df.append(np.concatenate((samples_sgd, samples[1:]), axis=0))
    grads_df.append(grads)    

Number of chains run: 100%|██████████| 11/11 [08:49<00:00, 48.15s/it]


In [8]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df)) #save output
runtime_df.to_csv("./out/lr_sgldcv_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lr_sgldcv_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lr_sgldcv_grads.csv", index=False)

### Running SGLD-CV-PS (0.001N) sampler

In [15]:
runtime_df=[]
samples_df = []
grads_df = []

In [16]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    theta_hat, samples_sgd, runtime_sgd = sgd.adam(key, gradf_0, gradf_i_batch, burnin, theta_0, x, y, n_batch, step_sgd, replacement=True)
    samples, grads, runtime, probs = sgldcvps.sgldcv_ps_sampler(subkey, gradf_0, gradf_i_batch, post_var, burnin, step_size, theta_hat, theta_hat, x, y, n_batch)
    runtime_df.append(np.concatenate((runtime_sgd, runtime_sgd[burnin-1]+runtime))) #join output
    samples_df.append(np.concatenate((samples_sgd, samples[1:]), axis=0))
    grads_df.append(grads) 

Number of chains run: 100%|██████████| 11/11 [27:17<00:00, 148.88s/it]


In [17]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lr_sgldcvps_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lr_sgldcvps_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lr_sgldcvps_grads.csv", index=False)